In [57]:
import pandas as pd
import numpy as np
from datetime import datetime
df=pd.read_csv('/home/duongnguyen/Downloads/sphist.csv')


In [58]:
#print(df.head(10) )
df["Date"]=pd.to_datetime(df["Date"])
df=df.sort_values(by=['Date'],ascending=True)
#print(df['Date'].iloc[:])  
df['day_5']=0
df['day_20']=0
df['day_50']=0
df.reset_index(inplace=True)
df.head(5)



,index,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_20,day_50
0,16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,0,0,0
1,16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,0,0,0
2,16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,0,0,0
3,16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,0,0,0
4,16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,0,0,0


In [59]:

'''
for index, _ in df.iterrows():
   # print(index)
    df.loc[index,'day_5']=df['Close'].iloc[(index-5):index].mean()
    #print(df['Close'].iloc[(index-5):index].mean())
    df.loc[index,'day_20']=df['Close'].iloc[(index-20):index].mean()
    df.loc[index,'day_50']=df['Close'].iloc[(index-50):index].mean()
  # loc is to refer ditrect to df, not it copy. loc is index by name, iloc by index, ['Close'] chain location may refer to the df view or it's copy
'''
#Another method which is much faster for calculate windown mean by rolling and shift
df['day_5'] = df['Close'].rolling(5).mean() #calculate mean in window of 5 values
df['day_5'] = df['day_5'].shift()           #shift value on this columns 1 index by default to remove current value in calculating 
df['day_20'] = df['Close'].rolling(20).mean()
df['day_20'] = df['day_20'].shift()
df['day_50'] = df['Close'].rolling(50).mean()
df['day_50'] = df['day_50'].shift()

df.head(60)


,index,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_20,day_50
0,16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN
1,16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN
2,16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN
3,16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN
4,16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN
5,16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900000,NaN,NaN
6,16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974000,NaN,NaN
7,16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022000,NaN,NaN
8,16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988000,NaN,NaN
9,16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926000,NaN,NaN


In [75]:

df['day_520']=df['day_5']-df['day_20']
df['day_550']=df['day_5']-df['day_50']
df['day_2050']=df['day_20']-df['day_50']
df['change']=df['Close']-df['Close'].shift(-1)
df.dropna(axis=0, inplace=True)
#train and test set
train=df[df['Date']<datetime(year=2013,month=1,day=1)]
test=df[df['Date']>=datetime(year=2013,month=1,day=1)]
train.tail(4)




,index,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_20,day_50,day_520,day_550,day_2050,change
15847,742,2012-12-26,1426.660034,1429.420044,1416.430054,1419.829956,2.285030e+09,1419.829956,1436.620019,1421.007995,1413.291797,15.612024,23.328223,7.716199,1.729980
15848,741,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1422.052496,1413.031597,9.175507,18.196406,9.020900,15.669922
15849,740,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1422.460992,1412.821797,5.224994,14.864189,9.639196,-23.759887
15850,739,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1421.784997,1412.067798,-2.350989,7.366211,9.717200,-36.230103


In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
LR_model=LinearRegression()
target=['Close']
features=train.columns.drop(['change','High','Close', 'Low', 'Open', 'Volume', 'Adj Close', 'Date','index', 'day_520', 'day_550', 'day_2050']).values          


target_change=['change']
features_change=train.columns.drop(['change','High','Close', 'Low', 'Open', 'Volume', 'Adj Close', 'Date','index', 'day_5','day_20','day_50']).values          
print(features,features_change)

['day_5' 'day_20' 'day_50'] ['day_520' 'day_550' 'day_2050']


In [79]:
LR_model.fit(train[features],train[target])
predict_close=LR_model.predict(test[features])
MAE=mean_absolute_error(predict_close,test[target])
MSE=mean_squared_error(predict_close,test[target])
print(MAE,MSE)

15.98984516079628 491.31871234755306


In [80]:
LR_model.fit(train[features_change],train[target_change])
predict_change=LR_model.predict(test[features_change])
#predict_close=LR_model.predict(test[features_change])+test['Close'].shift(-1)
MAE_change=mean_absolute_error(predict_change,test[target_change])
MSE_change=mean_squared_error(predict_change,test[target_change])
print(MAE_change,MSE_change)

10.970454258674552 227.4857191834147


In [86]:
predict_change=pd.DataFrame(predict_change)
predict_change

,0
0,-0.138587
1,-0.094049
2,-0.041531
3,0.026518
4,0.121787
5,0.164216
6,0.139520
7,0.142529
8,0.131425
9,0.122782


In [85]:
len(predict_change) 

738

In [84]:
len(test['Close'])

738

In [48]:

MAE_2=mean_absolute_error(test,test[target])
MSE_2=mean_squared_error(testtest[target])
print(MAE_2,MSE_2)

KeyError: 'predict'